In [ ]:
from catboost import CatBoostRegressor
from tpot import TPOTRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,LabelEncoder
from xgboost import XGBRegressor
from tensorflow.keras.layers import Normalization
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error as mae


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train=pd.read_csv("/kaggle/input/intra-aims-ml-challenge/train_set.csv")
test1=pd.read_csv("/kaggle/input/intra-aims-ml-challenge/test_set.csv")
test=pd.read_csv("/kaggle/input/intra-aims-ml-challenge/test_set.csv")

In [ ]:
train.describe(include='object')

In [ ]:
X=train[['Product_Code','Product_Weight','Fat_Category','Visibility_Score','Category_Type','Retail_Price','Store_Code','Year_Opened','Store_Size','Location_Class','Store_Category']]
Y=train.Total_Sales

In [ ]:
X.head()

In [ ]:
X['Year_Opened'] = 2024 - X['Year_Opened']

cat_cols = ['Product_Code','Fat_Category','Category_Type','Store_Code','Store_Size','Location_Class', 'Store_Category']  
num_cols = ['Product_Weight', 'Visibility_Score', 'Retail_Price', 'Year_Opened']  # Example numerical columns


cat_tf1 = Pipeline(steps=[
     
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='mean')) 
])




scaler = StandardScaler()

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()


X_scaled = scaler.fit_transform(X[['Visibility_Score']])

print(X_scaled)


label_encoder = LabelEncoder()
X['Fat_Category'] = label_encoder.fit_transform(X['Fat_Category'])
X['Store_Size'] = label_encoder.fit_transform(X['Store_Size'])


freq_encoding = X['Product_Code'].value_counts().to_dict()
X['Product_Code'] = X['Product_Code'].map(freq_encoding)

X['Visibility_Score'].fillna(X['Visibility_Score'].median(), inplace=True)


X['Product_Weight'] = X.groupby('Category_Type')['Product_Weight'].transform(lambda x: x.fillna(x.mean()))


preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', cat_tf1, cat_cols),  
        ('scaler', scaler, num_cols)  
    ]
)


X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
model = CatBoostRegressor(
    iterations=2400, 
    l2_leaf_reg=5,
    learning_rate=0.0035,            
    depth=5,
    early_stopping_rounds=50,                      
    loss_function="RMSE",         
    verbose=70                  
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('model', model)   ])             

pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_valid)

print("Mean Squared Error = ", mse(Y_valid, Y_pred))
print("Mean Absolute Error = ", mae(Y_valid, Y_pred), "\n")

In [ ]:
test["Year_Opened"]=2024-test["Year_Opened"]

test_pred=pipeline.predict(test)

In [ ]:
output=pd.DataFrame({'Id':test1.Id,'Total_Sales':test_pred})
output.to_csv('AnugyaAIMSSubmission.csv',index=False)